In [183]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import KFold


In [184]:
train_in = np.loadtxt('./Datasets/train_in.csv', 
                  delimiter=',',  
                  skiprows=0,      
                  dtype=float)       
test_in = np.loadtxt('./Datasets/test_in.csv', 
                  delimiter=',',   
                  skiprows=0,      
                  dtype=float)       
train_out = np.loadtxt('./Datasets/train_out.csv', 
                  delimiter=',',  
                  skiprows=0,      
                  dtype=float)       
test_out = np.loadtxt('./Datasets/test_out.csv', 
                  delimiter=',',   
                  skiprows=0,      
                  dtype=float)       

In [185]:
X_train, X_val, y_train, y_val = train_test_split(train_in, train_out, train_size=0.8, random_state=0)
X_test = test_in
y_test = test_out
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print('Inputs_train', )

(1365, 256) (342, 256) (1365,) (342,)
Inputs_train


In [186]:
# input dataset
def preprocessing(datasets):
    return np.hstack((datasets, np.ones(len(datasets)).reshape(len(datasets),1)))

In [187]:
def get_dummies(list, array):
    for i, each in zip(list, array):
        each[int(i)] = 1
    return array

In [188]:
# sigmoid function 
def sig(x):
    return (1/(1+np.exp(-x)))

def sig_prime(x):
    return (x * (1 - x))

In [189]:
Len_train = len(X_train)
Len_val = len(X_val)
Len_test = len(X_test)

X_train = preprocessing(X_train)
X_val = preprocessing(X_val)
X_test = preprocessing(X_test)

y_train_dummy = np.zeros(Len_train*10).reshape(Len_train, 10)
y_train_dummy = get_dummies(y_train, y_train_dummy)
y_val_dummy = np.zeros(Len_val*10).reshape(Len_val, 10)
y_val_dummy = get_dummies(y_val, y_val_dummy)
y_test_dummy = np.zeros(Len_test*10).reshape(Len_test, 10)
y_test_dummy = get_dummies(y_test, y_test_dummy)

In [190]:
kf = KFold(n_splits=5)

kfold_index_list = []

for cv_index_train, cv_index_test in kf.split(X_train):
    kfold_index_list.append((cv_index_train, cv_index_test))

In [305]:
def training(lr, w, x, y_dummy):
    for iter in range(1):
        for i in range(len(x)):
            y_hat = sig(np.dot(x[i], w))
            print(y_hat)
            break
            
            diff = y_dummy[i] - y_hat
            x_re = x[i].reshape(257,1)
            # print(y_hat)
            sp = sig_prime(y_hat) * diff
            w += lr * x_re * sp

        if iter % 10 == 0 :
            arg_y_hat = np.argmax(sig(np.dot(x, w)), axis=1)
            arg_y_dummy = np.argmax(y_dummy, axis=1)
            arg_diff = abs(arg_y_dummy - arg_y_hat)
            acc = (len(y_dummy)-np.sum(arg_diff>0))/len(y_dummy)
            # print(sum(arg_diff))
        if acc > 0.97:
            break
    return w

In [306]:
np.random.seed(2022)
w0 = np.random.random((257,10))
lr = 0.3
w = training(lr, w0, X_train, y_train_dummy)

[6.58508602e-26 2.19986935e-27 2.56338564e-25 4.66747100e-23
 2.46219685e-25 2.01273252e-22 1.12407051e-27 3.49367980e-26
 8.80330772e-26 1.48612205e-26]


In [193]:
# def training(lr, w, train_x, y_train_dummy):
#     for iter in range(100000):
#         y_hat = sig(np.dot(train_x, w))
#         # e = pow((y_train_dummy - y) * (y_train_dummy - y), 0.5)
#         diff = y_train_dummy - y_hat
#         # if iter % 100 == 0 :
#         #    print(sum(sum(abs(e))))
#         if sum(sum(abs(diff))) <= 1e-04:
#             # print(iter)
#             break
#         # updata w0(weight)
#         # print(w0.shape, train_x.shape, e.shape)
#         # print((1-w0) * w0)
#         # w0 += lr * np.dot(train_x.T, e) * ((1-w0) * w0)
#         w += lr * np.dot(train_x.T, diff)
#         # print(sum(sum(((1-w0) * w0))))
#     return w

In [194]:
def predict(lr, w, train_x, y_train_dummy):
    y_hat = sig(np.dot(train_x, w))
    return np.argmax(y_hat, axis=1)

In [195]:
len(y_train_dummy[kfold_index_list[1][1]])

273

In [234]:
W_experiments = np.zeros(257*10*4).reshape(4, 257,10)


In [267]:
def generate_w0(sections):
    np.random.seed(2022)
    W_experiments = np.zeros(257*10*len(sections)).reshape(len(sections), 257,10)
    index = 0
    for i in sections:
        # print(i[0], i[1])
        W_experiments[index] = np.random.uniform(i[0], i[1], size=(257,10))
        index += 1
    return W_experiments

In [308]:
# np.random.seed(2022)
# w0 = np.random.uniform(-0.3,0.3,size=(257,10))
sections = [(0, 1), (-0.5, 0.5), (-0.3, 0.3), (-0.1, 0.1)]
lr = 0.3
W_experiments = generate_w0(sections)


In [286]:
def generate_acc(w):
    pred_val = predict(lr, w, X_val, y_val_dummy)
    acc_val = accuracy_score(y_val, pred_val)

    pred_test = predict(lr, w, X_test, y_test_dummy)
    acc_test = accuracy_score(y_test, pred_test)

    return acc_val, acc_test

In [288]:
w_list = []
results = []
w_list.clear()
results.clear()
np.random.seed(2022)
w0 = np.random.random((257,10))
w = training(lr, w0, X_train, y_train_dummy)

pred_train = predict(lr, w, X_train, y_train_dummy)
acc_train = accuracy_score(y_train, pred_train)
acc_val, acc_test = generate_acc(w)

w_list.append(w)

print("Training with weights section: ", 'Completely random')
print("Accuracy train: ", acc_train, "Accuracy validation", acc_val, "Accuracy test", acc_test)

Training with weights section:  Completely random
Accuracy train:  0.18827838827838828 Accuracy validation 0.21637426900584794 Accuracy test 0.228


In [309]:
for w0_index in range(len(sections)):
    print(w0_index)
    w0 = W_experiments[w0_index]
    w = training(lr, w0, X_train, y_train_dummy)
    
    pred_train = predict(lr, w, X_train, y_train_dummy)
    acc_train = accuracy_score(y_train, pred_train)
    acc_val, acc_test = generate_acc(w)

    w_list.append(w)
    results.append((acc_train, acc_val, acc_test))
for i in range(len(sections)):
    acc, acc_val, acc_test = results[i]
    print("Training with weights section: ", sections[i])
    print("Accuracy train: ", acc, "Accuracy validation", acc_val, "Accuracy test", acc_test)



0
[6.58508602e-26 2.19986935e-27 2.56338564e-25 4.66747100e-23
 2.46219685e-25 2.01273252e-22 1.12407051e-27 3.49367980e-26
 8.80330772e-26 1.48612205e-26]
1
[8.37092592e-04 6.36181405e-02 1.06047226e-03 1.84816612e-01
 4.05882067e-03 4.32710131e-02 1.21598014e-02 1.37910674e-01
 3.54921376e-01 9.24581012e-01]
2
[0.94113812 0.28103678 0.82438899 0.1787882  0.98030265 0.04542156
 0.86962751 0.10038062 0.02523382 0.22078227]
3
[0.78453517 0.36183599 0.6208968  0.74105359 0.21007787 0.25580886
 0.20071424 0.60120015 0.65347593 0.52780036]
Training with weights section:  (0, 1)
Accuracy train:  0.9413919413919414 Accuracy validation 0.9502923976608187 Accuracy test 0.872
Training with weights section:  (-0.5, 0.5)
Accuracy train:  0.9560439560439561 Accuracy validation 0.9473684210526315 Accuracy test 0.882
Training with weights section:  (-0.3, 0.3)
Accuracy train:  0.9706959706959707 Accuracy validation 0.956140350877193 Accuracy test 0.883
Training with weights section:  (-0.1, 0.1)
Acc

In [292]:
w0 = np.zeros(257*10).reshape(257, 10)
w = training(lr, w0, X_train, y_train_dummy)

pred_train = predict(lr, w, X_train, y_train_dummy)
acc_train = accuracy_score(y_train, pred_train)
acc_val, acc_test = generate_acc(w)

w_list.append(w)

print("Training with weights section: ", '0, 0')
print("Accuracy train: ", acc_train, "Accuracy validation", acc_val, "Accuracy test", acc_test)

Training with weights section:  0, 0
Accuracy train:  0.9706959706959707 Accuracy validation 0.956140350877193 Accuracy test 0.879


In [294]:
len(w_list)

11

In [254]:
def best_weight():
    return 0

In [300]:
np.random.seed(2022)
w0 = np.random.uniform(-0.3, 0.3, size=(257,10))
lr = 0.3
w_list.clear()
results.clear()
for i in range(5):
    w = training(lr, w0, X_train[kfold_index_list[i][0]], y_train_dummy[kfold_index_list[i][0]])

    pred_train = predict(lr, w, X_train[kfold_index_list[i][1]], y_train_dummy[kfold_index_list[i][1]])
    acc_train = accuracy_score(y_train[kfold_index_list[i][1]], pred_train)

    pred_val = predict(lr, w, X_val, y_val_dummy)
    acc_val = accuracy_score(y_val, pred_val)

    pred_test = predict(lr, w, X_test, y_test_dummy)
    acc_test = accuracy_score(y_test, pred_test)

    w_list.append(w)
    results.append((acc_train, acc_val, acc_test))

for acc, acc_val, acc_test in results:
    print("Accuracy train: ", acc, "Accuracy validation", acc_val, "Accuracy test", acc_test)


Accuracy train:  0.9413919413919414 Accuracy validation 0.9502923976608187 Accuracy test 0.872
Accuracy train:  0.9560439560439561 Accuracy validation 0.9473684210526315 Accuracy test 0.882
Accuracy train:  0.9706959706959707 Accuracy validation 0.956140350877193 Accuracy test 0.883
Accuracy train:  0.9816849816849816 Accuracy validation 0.9473684210526315 Accuracy test 0.881
Accuracy train:  0.9816849816849816 Accuracy validation 0.9502923976608187 Accuracy test 0.879


In [298]:
len(w_list)

5

In [301]:
## confusion Test
np.random.seed(2022)
w0 = np.random.uniform(-0.3, 0.3, size=(257,10))
w = training(lr, w0, X_train, y_train_dummy)
confusion_matrix(predict(lr, w, X_test, y_test_dummy), y_test)

array([[215,   0,   3,   3,   2,   3,   2,   0,   3,   0],
       [  0, 113,   0,   0,   1,   0,   0,   0,   2,   1],
       [  1,   0,  80,   2,   0,   0,   0,   1,   3,   0],
       [  1,   0,   2,  67,   0,  11,   1,   1,   4,   0],
       [  3,   2,   5,   1,  76,   2,   1,   3,   1,   3],
       [  1,   0,   0,   1,   2,  35,   0,   0,   2,   2],
       [  1,   3,   2,   0,   2,   0,  86,   0,   0,   0],
       [  0,   2,   2,   1,   1,   2,   0,  56,   4,   2],
       [  1,   0,   7,   3,   0,   0,   0,   0,  73,   1],
       [  1,   1,   0,   1,   2,   2,   0,   3,   0,  79]])

In [199]:
# w0 = np.random.random((257,10))
# lr = 0.3
# w = training(lr, w0, X_train, y_train_dummy)